# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""16973262119504954068""","""../../../../data/SampleHDS.jso…","""16769757323054342538""",7.9897e18,"""7378456768465453339""","""3237589221449260092""",false
"""15203391337876692327""","""../../../../data/SampleHDS.jso…","""8820581660925778813""",5.0455e18,"""7378456768465453339""","""10987607310530755593""",true
"""1410635593663362419""","""../../../../data/SampleHDS.jso…","""5472759114985300282""",4.6870e18,"""15306289255310134823""","""3237589221449260092""",false
"""3490043960559986895""","""../../../../data/SampleHDS.jso…","""8820581660925778813""",1.1310e19,"""5354054396074701478""","""3237589221449260092""",false
"""17841821664015346765""","""../../../../data/SampleHDS.jso…","""16769757323054342538""",1.6129e19,"""5354054396074701478""","""17506178964919490453""",true
"""1764248824136647765""","""../../../../data/SampleHDS.jso…","""5472759114985300282""",8.7445e18,"""17158305345367556538""",null,false
"""1055750501475932723""","""../../../../data/SampleHDS.jso…","""5472759114985300282""",1.5038e19,"""5354054396074701478""","""17506178964919490453""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""14139342854704801436""","""../../../../data/SampleHDS.jso…","""2553170829423693464""",5.5445e18,"""FirstMortgage30yr""","""9444523542945246950""","""Rejected"""
"""774452191720043904""","""../../../../data/SampleHDS.jso…","""11611196088168454847""",7.8694e18,"""FirstMortgage30yr""","""13706379782120304260""","""Accepted"""
"""8062875547133503190""","""../../../../data/SampleHDS.jso…","""7902914339965925500""",8.8373e17,"""MoneyMarketSavingsAccount""","""9444523542945246950""","""Rejected"""
"""9506024330188948219""","""../../../../data/SampleHDS.jso…","""11611196088168454847""",7.8056e18,"""BasicChecking""","""9444523542945246950""","""Rejected"""
"""9415711733599794357""","""../../../../data/SampleHDS.jso…","""2553170829423693464""",1.2800e19,"""BasicChecking""","""7776071456149207305""","""Accepted"""
"""4577083987942810763""","""../../../../data/SampleHDS.jso…","""7902914339965925500""",2.0074e18,"""UPlusFinPersonal""",null,"""Rejected"""
"""548257719147493657""","""../../../../data/SampleHDS.jso…","""7902914339965925500""",9.6629e18,"""BasicChecking""","""7776071456149207305""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""5789410699764732053""","""../../../../data/SampleHDS.jso…","""17774495357201775844""",8.7165e17,"""FirstMortgage30yr""","""9145986927403529652""","""Rejected"""
"""260293450102005196""","""../../../../data/SampleHDS.jso…","""2757563940162061772""",7.6736e18,"""FirstMortgage30yr""","""10807676693282446406""","""Accepted"""
"""17138415298265998207""","""../../../../data/SampleHDS.jso…","""5242581423834464452""",1.0035e19,"""MoneyMarketSavingsAccount""","""9145986927403529652""","""Rejected"""
"""6840943039295403752""","""../../../../data/SampleHDS.jso…","""2757563940162061772""",1.3343e19,"""BasicChecking""","""9145986927403529652""","""Rejected"""
"""18081428790883115077""","""../../../../data/SampleHDS.jso…","""17774495357201775844""",8.9795e18,"""BasicChecking""","""612317669878353385""","""Accepted"""
"""1612215997988902543""","""../../../../data/SampleHDS.jso…","""5242581423834464452""",1.3785e19,"""UPlusFinPersonal""",null,"""Rejected"""
"""5593519537167506013""","""../../../../data/SampleHDS.jso…","""5242581423834464452""",9.3686e17,"""BasicChecking""","""612317669878353385""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""16432589228082651687""","""13060153580748269977""",8.4186e18,"""FirstMortgage30yr""","""8553180119775720838""","""Rejected"""
"""14943685715148380089""","""6122781821376309222""",1.6719e19,"""FirstMortgage30yr""","""9920794390560520769""","""Accepted"""
"""14221851940079056010""","""11576141859736881560""",1.3276e19,"""MoneyMarketSavingsAccount""","""8553180119775720838""","""Rejected"""
"""3681168313723602867""","""6122781821376309222""",8.0982e18,"""BasicChecking""","""8553180119775720838""","""Rejected"""
"""2584503479983220052""","""13060153580748269977""",4.0588e18,"""BasicChecking""","""7784849784146206540""","""Accepted"""
"""12405793527474579255""","""11576141859736881560""",1.2600e19,"""UPlusFinPersonal""",null,"""Rejected"""
"""15029403341541610783""","""11576141859736881560""",3.4771e18,"""BasicChecking""","""7784849784146206540""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'